In [1]:
from pyspark.sql import *
import pandas as pd
from time import time
from GetPath import *

In [2]:
# Config
input_path='D:/output/chenbingying/B2/'
output_path='D:/Output/chenbingying/Check/'

In [3]:
def Check_B2(input_path,output_path):
    t_start=time()
    spark = SparkSession.builder.appName('B2_Check').config('spark.dynamicAllocation.enabled','true').config('spark.debug.maxToStringFields', '100').config('spark.sql.execution.arrow.enable','true').config('spark.executor.memory','48g').config('spark.driver.memory', '48g').config('spark.core.connection.ack.wait.timeout','36000s').config('spark.executor.heartbeatInterval','36000s').config('spark.network.timeout', '50000s').config('spark.rpc.lookupTimeout', '5000s').config('spark.shuffle.io.connectionTimeout', '50000s').getOrCreate()
    result_list=[]
    namelist=[]
    filelist=GetPath(input_path)
    for file in filelist:
        print("正在处理 ",file)
        start=time()
        spark.read.option('inferSchema','true').option('header','true').csv(file).createOrReplaceTempView('B2_Raw')
        print('读入数据耗时：%.2f秒'%(time()-start))
        start=time()
        result=spark.sql('''select 
            count(*) as TOTAL
            ,count(ID_NUMBER) as ID_NUMBER
            ,count(AGE) as AGE
            ,count(GENDER) as GENDER
            ,count(VISITING_DATE) as VISITING_DATE 
            ,count(DISEASE_DIAGNOSIS_CODE) as DISEASE_DIAGNOSIS_CODE
            ,count(VISITING_TYPE_CODE) as VISITING_TYPE_CODE
            ,count(VISITING_TYPE_NAME) as VISITING_TYPE_NAME
            ,count(ICD_CODE) as ICD_CODE
            ,count(CITY) as CITY
            ,count(DISTRICT) as DISTRICT
            ,sum(case when MEDICAL_INSTITUTION_CODE is not null and FILE_NUMBER is not null then 1 else 0 end) as MF
            ,sum(case when CARD_TYPE is not null and CARD_NUMBER is not null then 1 else 0 end) as CC
            from B2_Raw
        ''').toPandas()
        for i in result.columns:
            if(i!='TOTAL'):
                result.insert(result.columns.get_loc(i)+1,i+'_RATE',result[i]/result['TOTAL']*100)
        result_list.append(result)
        print('处理数据耗时：%.2f秒'%(time()-start))
        namelist.append(file.split('/')[-2:-1][0])
    df=pd.concat(result_list)
    df.insert(0,'FILE',namelist)
    df.columns=['文件名', '数据总量','ID_NUMBER非空数据量','ID_NUMBER完整率','AGE非空数据量','AGE完整率','GENDER非空数据量','GENDER完整率','VISITING_DATE非空数据量','VISITING_DATE完整率','DISEASE_DIAGNOSIS_CODE非空数据量','DISEASE_DIAGNOSIS_CODE完整率','VISITING_TYPE_CODE非空数据量','VISITING_TYPE_CODE完整率','VISITING_TYPE_NAME非空数据量','VISITING_TYPE_NAME完整率','ICD_CODE非空数据量','ICD_CODE完整率','CITY非空数据量','CITY完整率','DISTRICT非空数据量','DISTRICT完整率','MEDICAL_INSTITUTION_CODE,FILE_NUMBER字段组合非空个数==2数据量','MEDICAL_INSTITUTION_CODE,FILE_NUMBER字段组合非空个数==2占比','CARD_TYPE,CARD_NUMBER字段组合非空个数==2数据量','CARD_TYPE,CARD_NUMBER字段组合非空个数==2占比']
    df.to_csv(output_path+'B2统计.csv',index=None)
    spark.catalog.dropTempView("B2_Raw")
    print(f'已输出至：{output_path}B2统计.csv','\n总耗时：%.2f秒'%(time()-t_start))

In [4]:
if __name__=="__main__":
    Check_B2(input_path,output_path)

正在处理  D:/output/zhuhao/B2/tb_cis_op_visiting_record_201401_201806.csv
读入数据耗时：71.79秒
处理数据耗时：86.67秒
正在处理  D:/output/zhuhao/B2/tb_cis_op_visiting_record_201807_201912.csv
读入数据耗时：54.13秒
处理数据耗时：66.39秒
正在处理  D:/output/zhuhao/B2/tb_cis_op_visiting_record_202001_202012.csv
读入数据耗时：59.81秒
处理数据耗时：75.31秒
正在处理  D:/output/zhuhao/B2/tb_cis_op_visiting_record_202101_202112.csv
读入数据耗时：55.38秒
处理数据耗时：73.74秒
正在处理  D:/output/zhuhao/B2/tb_cis_op_visiting_record_202201_202205.csv
读入数据耗时：30.25秒
处理数据耗时：39.84秒
已输出至：D:/Output/zhuhao/Check/B2统计.csv 
总耗时：616.80秒
